In [1]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.loader import DataLoader
import sys
import os
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)
from simulations.projectile_trajectory_2d import projectile_trajectory_2d
from gnn_model.graph_structure_from_trajecotry import node_data_list
from gnn_model.message_passing_MLP import GNN_MLP
from gnn_model.train_model import train_model

## Calling simulations to create data

In [4]:
# loading data
trajectory = projectile_trajectory_2d(2, 0.01, 35, 100)
graph_data_trajectory = node_data_list(trajectory, self_loop=True, complete_graph=False)
print(graph_data_trajectory)
unique_graphs = set([str(data) for data in graph_data_trajectory])
print(len(unique_graphs))
# initialising model
# model = GNN_MLP(in_channels=2, out_channels=2, hidden_channels=128)
# train_model(model, train_data = graph_data_trajectory, lr=0.01, epochs=100)

[Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]), Data(x=[1

/Users/jacobtoot/Documents/GitHub/DeepLearningPH/gnn_model/graph_structure_from_trajecotry.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float)
